In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

In [2]:
df_bt = pd.read_csv('blocks_timestamps.csv') # Timestamps for each block
df_om = pd.read_csv('ordermatches_all.csv')  # Order details for each order (block)
df_pp = pd.read_csv('pepecash_prices.csv')   # USD value of pepecash over time (timeblock)
df_xp = pd.read_csv('xcp_prices.csv')        # USD value of xcp over time (timeblock)

In [3]:
df_bt

,Block,Timestamp
0,428925,1473387721
1,428926,1473388413
2,428927,1473391665
3,428928,1473391811
4,428929,1473393165
5,428930,1473394241
6,428931,1473396044
7,428932,1473397906
8,428933,1473398672
9,428934,1473399792


In [4]:
df_om

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity
0,428940,RAREPEPE,1.00,XCP,2.000
1,429221,SHITCOINCARD,230.00,XCP,13.800
2,429221,RAREPEPE,10.00,XCP,15.000
3,429228,SHITCOINCARD,3.00,XCP,0.180
4,429228,RAREPEPE,1.00,XCP,1.500
5,429398,SHITCOINCARD,12.00,XCP,0.720
6,429398,RAREPEPE,2.00,XCP,3.000
7,429403,RAREPEPE,1.00,XCP,1.500
8,429403,SHITCOINCARD,755.00,XCP,45.300
9,429466,RAREPEPE,1.00,XCP,1.500


In [5]:
df_pp

,Timestamp,Price
0,1517558400,0.052494
1,1517472000,0.041177
2,1517385600,0.062411
3,1517299200,0.064185
4,1517212800,0.071138
5,1517126400,0.073951
6,1517040000,0.074749
7,1516953600,0.068029
8,1516867200,0.083559
9,1516780800,0.080997


In [6]:
df_xp

,Timestamp,Price
0,1517558400,32.75
1,1517472000,33.52
2,1517385600,43.16
3,1517299200,44.83
4,1517212800,53.25
5,1517126400,59.47
6,1517040000,56.05
7,1516953600,50.57
8,1516867200,56.33
9,1516780800,59.29


In [7]:
# Need to throw out all data that isn't xcp or pepecash value
# JK no we don't
# df_om.drop(df_om[df_om.ForwardAsset != ('PEPECASH' or 'XCP')].index, inplace=True)

In [8]:
#df_om_merge = pd.concat([df_om, df_bt], axis=1, join='inner')
df_om_merge = pd.merge(df_om, df_bt, on='Block', how='inner')


In [9]:
#df_om.insert
#df_om_merge = pd.merge(df_om, df_bt)
#df_om_merge = pd.concat([df_om, df_bt], on='Block', axis=1)
df_om_merge

,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,428940,RAREPEPE,1.0,XCP,2.00,1473403689
1,429221,SHITCOINCARD,230.0,XCP,13.80,1473561677
2,429221,RAREPEPE,10.0,XCP,15.00,1473561677
3,429228,SHITCOINCARD,3.0,XCP,0.18,1473567368
4,429228,RAREPEPE,1.0,XCP,1.50,1473567368
5,429398,SHITCOINCARD,12.0,XCP,0.72,1473654430
6,429398,RAREPEPE,2.0,XCP,3.00,1473654430
7,429403,RAREPEPE,1.0,XCP,1.50,1473656361
8,429403,SHITCOINCARD,755.0,XCP,45.30,1473656361
9,429466,RAREPEPE,1.0,XCP,1.50,1473687080


In [10]:
df_om_merge.columns

Index(['Block', 'ForwardAsset', 'ForwardQuantity', 'BackwardAsset',
       'BackwardQuantity', 'Timestamp'],
      dtype='object')

In [11]:
'''#for row in df_om_merge:
df_om_merge2 = df_om_merge
for x, y in df_om_merge2.iterrows():
    check = False
    i = 0
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):
        while check != True or i<519:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
        i=0
        
    # PEPECASH Conversion
        check = False
    elif (df_om_merge2.loc[x, 'BackwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.BackwardAsset[x] = 'USD'
            i=i+1
        i=0
        check = False
    else:#(df_om_merge2.loc[x, 'ForwardAsset'] == 'PEPECASH'):
        while check != True or i<490:
            if(df_om_merge2.loc[x, 'Timestamp'] < df_pp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_pp.loc[i+1, 'Timestamp']):
                check = True
                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_pp.Price[i+1]
                df_om_merge2.ForwardAsset[x] = 'USD'
            i=i+1
           
        
        #for label, row in df_xp.iterrows():
                #print(label)
                #print(row)
                #if(ts >= df_xp.Timestamp[x] and ts < df_xp.Timestamp[x+1]):
                    #print(x)
                    #df_om_merge.BackwardQuantity[i] = df_xp.Price[x] * df_om_merge.BackwardQuantity[i]
                    #df_om_merge.BackwardAsset = 'USD'
                
#print(df_om_merge.Timestamp)
'''

"#for row in df_om_merge:\ndf_om_merge2 = df_om_merge\nfor x, y in df_om_merge2.iterrows():\n    check = False\n    i = 0\n    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.BackwardQuantity[x] = df_om_merge2.BackwardQuantity[x] * df_xp.Price[i+1]\n                df_om_merge2.BackwardAsset[x] = 'USD'\n            i=i+1\n        i=0\n        check = False\n    elif (df_om_merge2.loc[x, 'ForwardAsset'] == 'XCP'):\n        while check != True or i<519:\n            if(df_om_merge2.loc[x, 'Timestamp'] < df_xp.loc[i, 'Timestamp'] and df_om_merge2.loc[x, 'Timestamp'] >= df_xp.loc[i+1, 'Timestamp']):\n                check = True\n                df_om_merge2.ForwardQuantity[x] = df_om_merge2.ForwardQuantity[x] * df_xp.Price[i+1]\n               

In [12]:
# This dataset now has the USD equivalent of the XCP values and PEPECASH
df_om_merge2 = pd.read_csv('ordermatches_USD.csv')   
df_om_merge2

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
5,5,429398,SHITCOINCARD,12.000000,USD,2.491200,1473654430
6,6,429398,RAREPEPE,2.000000,USD,10.380000,1473654430
7,7,429403,RAREPEPE,1.000000,USD,5.190000,1473656361
8,8,429403,SHITCOINCARD,755.000000,USD,156.738000,1473656361
9,9,429466,RAREPEPE,1.000000,USD,5.655000,1473687080


In [13]:
'''#df_om_merge2.to_csv(r'C:\Programming\cmpe188\rarepepes\ordermatches_USD.csv')
for x, y in df_om_merge2.iterrows():
    if(df_om_merge2.loc[x, 'BackwardAsset'] == 'USD'):
        df_om_merge2['PricePerPepe'] = df_om_merge2['BackwardQuantity'] / df_om_merge2['ForwardQuantity']
    else:
        df_om_merge2['PricePerPepe'] = df_om_merge2['ForwardQuantity'] / df_om_merge2['BackwardQuantity']
        '''
df_om_merge2 = pd.read_csv('ordermatches_USD.csv')  # Order details for each order (block)


In [14]:
df_om_merge2

,Unnamed: 0,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp
0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689
1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677
2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677
3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368
4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368
5,5,429398,SHITCOINCARD,12.000000,USD,2.491200,1473654430
6,6,429398,RAREPEPE,2.000000,USD,10.380000,1473654430
7,7,429403,RAREPEPE,1.000000,USD,5.190000,1473656361
8,8,429403,SHITCOINCARD,755.000000,USD,156.738000,1473656361
9,9,429466,RAREPEPE,1.000000,USD,5.655000,1473687080


In [15]:
#df_om_merge2.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\PPP.csv')
df_om_merge3 = pd.read_csv('PPP.csv')  
df_om_merge3

,Unnamed: 0,Unnamed: 0.1,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,0,0,428940,RAREPEPE,1.000000,USD,6.660000,1473403689,6.660000
1,1,1,429221,SHITCOINCARD,230.000000,USD,44.988000,1473561677,0.195600
2,2,2,429221,RAREPEPE,10.000000,USD,48.900000,1473561677,4.890000
3,3,3,429228,SHITCOINCARD,3.000000,USD,0.586800,1473567368,0.195600
4,4,4,429228,RAREPEPE,1.000000,USD,4.890000,1473567368,4.890000
5,5,5,429398,SHITCOINCARD,12.000000,USD,2.491200,1473654430,0.207600
6,6,6,429398,RAREPEPE,2.000000,USD,10.380000,1473654430,5.190000
7,7,7,429403,RAREPEPE,1.000000,USD,5.190000,1473656361,5.190000
8,8,8,429403,SHITCOINCARD,755.000000,USD,156.738000,1473656361,0.207600
9,9,9,429466,RAREPEPE,1.000000,USD,5.655000,1473687080,5.655000


In [16]:
'''for x, y in df_om_merge3.iterrows():
    if(df_om_merge3.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge3.ForwardQuantity[x], df_om_merge3.BackwardQuantity[x] = df_om_merge3.BackwardQuantity[x], df_om_merge3.ForwardQuantity[x]
df_om_merge3
'''
df_om_merge3 = pd.read_csv('CardLeftPriceRight.csv')  
df_om_merge3

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,0,0,0,428940,RAREPEPE,1.0,USD,6.660000,1473403689,6.660000
1,1,1,1,429221,SHITCOINCARD,230.0,USD,44.988000,1473561677,0.195600
2,2,2,2,429221,RAREPEPE,10.0,USD,48.900000,1473561677,4.890000
3,3,3,3,429228,SHITCOINCARD,3.0,USD,0.586800,1473567368,0.195600
4,4,4,4,429228,RAREPEPE,1.0,USD,4.890000,1473567368,4.890000
5,5,5,5,429398,SHITCOINCARD,12.0,USD,2.491200,1473654430,0.207600
6,6,6,6,429398,RAREPEPE,2.0,USD,10.380000,1473654430,5.190000
7,7,7,7,429403,RAREPEPE,1.0,USD,5.190000,1473656361,5.190000
8,8,8,8,429403,SHITCOINCARD,755.0,USD,156.738000,1473656361,0.207600
9,9,9,9,429466,RAREPEPE,1.0,USD,5.655000,1473687080,5.655000


In [17]:
'''for x, y in df_om_merge3.iterrows():
    if(df_om_merge3.loc[x, 'ForwardAsset'] == 'USD'):
        df_om_merge3.ForwardAsset[x], df_om_merge3.BackwardAsset[x] = df_om_merge3.BackwardAsset[x], df_om_merge3.ForwardAsset[x]
df_om_merge3
'''

"for x, y in df_om_merge3.iterrows():\n    if(df_om_merge3.loc[x, 'ForwardAsset'] == 'USD'):\n        df_om_merge3.ForwardAsset[x], df_om_merge3.BackwardAsset[x] = df_om_merge3.BackwardAsset[x], df_om_merge3.ForwardAsset[x]\ndf_om_merge3\n"

In [30]:
#df_om_merge3.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\CardLeftPriceRight.csv')
'''
df_om_merge4 = df_om_merge3
sorted = df_om_merge4.sort_values('ForwardAsset')
#df_om_merge4['ForwardAsset'].sort_values()
sorted
'''

"\ndf_om_merge4 = df_om_merge3\nsorted = df_om_merge4.sort_values('ForwardAsset')\n#df_om_merge4['ForwardAsset'].sort_values()\nsorted\n"

In [32]:
df_om_merge5 = pd.read_csv('SortCards.csv')
df_om_merge5

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,3349,3349,3349,3349,439210,BACHELORPEPE,1.0,USD,8.840000,1479306308,8.840000
1,4817,4817,4817,4817,446214,BACHELORPEPE,1.0,USD,38.400000,1483329596,38.400000
2,1783,1783,1783,1783,434852,BACHELORPEPE,1.0,USD,12.450000,1476797843,0.080321
3,1396,1396,1396,1396,434445,BACHELORPEPE,1.0,USD,17.040000,1476546441,17.040000
4,17129,17129,17129,17129,470549,BACHELORPEPE,1.0,USD,261.060000,1497035519,261.060000
5,20900,20900,20900,20900,477251,BACHELORPEPE,1.0,USD,333.420000,1500852631,333.420000
6,19865,19865,19865,19865,476883,BACHELORPEPE,1.0,USD,287.700000,1500657195,287.700000
7,19864,19864,19864,19864,476883,BACHELORPEPE,1.0,USD,287.700000,1500657195,287.700000
8,10456,10456,10456,10456,455933,BACHELORPEPE,1.0,USD,29.000000,1488760516,0.034483
9,22451,22451,22451,22451,480690,BACKTOPEPE,1.0,USD,5.316800,1502829416,5.316800


In [34]:
len(df_om_merge5['ForwardAsset'].unique())

1468

In [36]:
df_om_merge5.drop(df_om_merge5.columns[df_om_merge5.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)


,Block,ForwardAsset,ForwardQuantity,BackwardAsset,BackwardQuantity,Timestamp,PricePerPepe
0,439210,BACHELORPEPE,1.0,USD,8.840000,1479306308,8.840000
1,446214,BACHELORPEPE,1.0,USD,38.400000,1483329596,38.400000
2,434852,BACHELORPEPE,1.0,USD,12.450000,1476797843,0.080321
3,434445,BACHELORPEPE,1.0,USD,17.040000,1476546441,17.040000
4,470549,BACHELORPEPE,1.0,USD,261.060000,1497035519,261.060000
5,477251,BACHELORPEPE,1.0,USD,333.420000,1500852631,333.420000
6,476883,BACHELORPEPE,1.0,USD,287.700000,1500657195,287.700000
7,476883,BACHELORPEPE,1.0,USD,287.700000,1500657195,287.700000
8,455933,BACHELORPEPE,1.0,USD,29.000000,1488760516,0.034483
9,480690,BACKTOPEPE,1.0,USD,5.316800,1502829416,5.316800


In [37]:
#df_om_merge5.to_csv(r'C:\Users\Zachary Nguyen\Desktop\188 Project\rarepepes\SortCards.csv')